In [17]:
import numpy as np
import cv2
import os
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.preprocessing.image import img_to_array

In [18]:
# Define the directory containing your images
image_dir ='./data'

# Create an ImageDataGenerator for preprocessing
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,  # rescale pixel values to [0, 1]
    rotation_range=20,  # randomly rotate images by up to 20 degrees
    width_shift_range=0.1,  # randomly shift images horizontally by up to 10%
    height_shift_range=0.1,  # randomly shift images vertically by up to 10%
    shear_range=0.2,  # randomly apply shearing transformations
    zoom_range=0.2,  # randomly zoom into images
    horizontal_flip=True,  # randomly flip images horizontally
    fill_mode='nearest'  # fill in missing pixels using the nearest available value
)

# Load images from the directory
image_data = datagen.flow_from_directory(
    image_dir,
    target_size=(224, 224),  # resize images to 224x224
    batch_size=32,
    class_mode=None,  # set to None since there are no labels
    shuffle=False  # disable shuffling to keep images in order
)


Found 6432 images belonging to 2 classes.


In [19]:
# Preprocess the images (resize, normalize, etc.)
def preprocess_images_from_subdirectories(root_dir):
    preprocessed_images = []
    model = VGG16(weights='imagenet', include_top=False)
    
    # Iterate over subdirectories
    for class_dir in os.listdir(root_dir):
        class_path = os.path.join(root_dir, class_dir)
        
        # Check if it's a directory
        if os.path.isdir(class_path):
            # Iterate over files in the subdirectory
            for filename in os.listdir(class_path):
                if filename.endswith('.jpg') or filename.endswith('.png'):
                    # Load image
                    image_path = os.path.join(class_path, filename)
                    image = cv2.imread(image_path)
                    
                    # Resize image
                    image = cv2.resize(image, (224, 224))
                    
                    # Convert image to array and preprocess for VGG16
                    image = img_to_array(image)
                    image = np.expand_dims(image, axis=0)
                    image = image / 255.0
                    
                    # Extract features using VGG16 model
                    features = model.predict(image)
                    features = features.flatten()
                    
                    # Append preprocessed features to list
                    preprocessed_images.append(features)
    
    # Convert list to numpy array
    X = np.array(preprocessed_images)
    
    return X

# Example usage:
root_dir = './Data'
X_train_preprocessed = preprocess_images_from_subdirectories(os.path.join(root_dir, 'train'))
X_test_preprocessed = preprocess_images_from_subdirectories(os.path.join(root_dir, 'test'))

1/1 [==============================] - 1s 622ms/step


In [26]:
#  Load pre-trained VGG16 model (without the top Dense layers)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


In [95]:
print("Shape of X_train_preprocessed:", X_train_preprocessed.shape)
print("Size of X_train_preprocessed:", X_train_preprocessed.size)

print("Shape of X_test_preprocessed:", X_test_preprocessed.shape)
print("Size of X_test_preprocessed:", X_test_preprocessed.size)


Shape of X_train_preprocessed: (5144, 25088)
Size of X_train_preprocessed: 129052672
Shape of X_test_preprocessed: (1288, 25088)
Size of X_test_preprocessed: 32313344


In [94]:
for layer in base_model.layers:
    layer.trainable = False
    
# Extract features using VGG16
X_train_features = base_model.predict(X_train_preprocessed)
X_test_features = base_model.predict(X_test_preprocessed)

# Reshape features into sequences
sequence_length =1 #X_train_features.shape[1]
num_features = 25088#X_train_features.shape[2] * X_train_features.shape[3]  # Flatten the spatial dimensions
X_train_sequences = X_train_features.reshape(-1, sequence_length, num_features)
X_test_sequences = X_test_features.reshape(-1, sequence_length, num_features)


# # Reshape features into sequences
# sequence_length = 1  # Define the desired sequence length
# num_features_per_sequence = 25088  # Define the number of features per sequence

# # Calculate the total number of sequences
# total_samples, num_features = X_train_preprocessed.shape
# total_elements = total_samples * num_features

# # Calculate the number of sequences that can be formed
# num_sequences = total_elements // (sequence_length * num_features_per_sequence)

# # Calculate the total number of elements needed for reshaping
# total_elements_needed = num_sequences * sequence_length * num_features_per_sequence

# # Trim the array to have an exact number of elements required for reshaping into sequences
# X_train_trimmed = X_train_preprocessed[:total_elements_needed]

# # Reshape the array
# X_train_sequences = X_train_trimmed.reshape(num_sequences, sequence_length, num_features_per_sequence)

# X_test_sequences = X_test_preprocessed.reshape(-1, sequence_length, num_features)




ValueError: in user code:

    File "c:\Users\Devender\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 2416, in predict_function  *
        return step_function(self, iterator)
    File "c:\Users\Devender\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 2401, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Devender\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 2389, in run_step  **
        outputs = model.predict_step(data)
    File "c:\Users\Devender\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 2357, in predict_step
        return self(x, training=False)
    File "c:\Users\Devender\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filter

In [90]:
# Define the LSTM model
model = Sequential([
    LSTM(64, input_shape=(sequence_length, num_features)),
    Dense(1, activation='sigmoid')
])

In [91]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [92]:
# Train the model
model.fit(X_train_sequences, epochs=10)

Epoch 1/10


ValueError: in user code:

    File "c:\Users\Devender\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 1377, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\Devender\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 1360, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Devender\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 1349, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\Devender\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 1128, in train_step
        self._validate_target_and_loss(y, loss)
    File "c:\Users\Devender\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 1082, in _validate_target_and_loss
        raise ValueError(


In [ ]:
# Evaluate the model on the testing set
model.evaluate(X_test_sequences)